# 


# ERA5 Magdalena River Basin Daily: Volume Aggregation Error

## 0. Import Libraries and define functions

In [ ]:
import pandas as pd
import numpy as np
import netCDF4 as nc
from math import sqrt
import datetime as dt
import scipy
import warnings # Ignore not important warnings
import matplotlib.pyplot as plt # Basic Plotting library
from skimage import measure
warnings.filterwarnings("ignore")
import plotly.graph_objects as go

## 1. Load Data

#### **Local Data NetCDF(IDW) - 0.1**

In [ ]:
# NC path
idw10_path="Data/Local IDW/IDW_daily_010.nc" 
# Check NetCDF Time
idw10nc = nc.Dataset(idw10_path)
time_unit=idw10nc.variables['time'].units 
time_cal=idw10nc.variables['time'].calendar
time_valh=idw10nc.variables['time'][:]
time_his=nc.num2date(time_valh,units=time_unit,calendar=time_cal)
time_his=sorted([dt.datetime.strptime(k.strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') for k in time_his])
# NetCDF grid data
nc_lat_idw010=idw10nc.variables['latitude'][:]
nc_lon_idw010=idw10nc.variables['longitude'][:]
if  (nc_lon_idw010<0).any():
    nc_lon_idw010=nc_lon_idw010+360   
# Precipitation variable name
var_type2=list(idw10nc.variables.keys())[-1]    
# Load data
idw10=idw10nc.variables[var_type2][:,:,:].data

#### **ERA Land Data**

In [ ]:
# NC path
eraland_path="Data/ERALand NC/precipitation_eraland_d2.nc"
# Check NetCDF Time
eralandnc = nc.Dataset(eraland_path)
time_unit=eralandnc.variables['time'].units 
time_cal=eralandnc.variables['time'].calendar
time_valh=eralandnc.variables['time'][:]
time_his=nc.num2date(time_valh,units=time_unit,calendar=time_cal)
time_his=sorted([dt.datetime.strptime(k.strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') for k in time_his])
# NetCDF grid data
nc_lat_eraland=eralandnc.variables['latitude'][:]
nc_lon_eraland=eralandnc.variables['longitude'][:]
if  (nc_lon_eraland<0).any():
    nc_lon_eraland=nc_lon_eraland+360   
# Precipitation variable name
var_type4=list(eralandnc.variables.keys())[-1]    
# Load data
eraland=eralandnc.variables[var_type4][:,:,:].data
eralandnc.close()

#### **Basin Mask 0.10**

In [ ]:
### Basin Mask 0.10 (.nc file)
rain_nc="Data/Magdalena/GIS/Basin/Basin_ext_010.nc"
basin = nc.Dataset(rain_nc)
basin_lat_010=basin.variables['lat'][:]
basin_lon_010=basin.variables['lon'][:]
var_type=list(basin.variables.keys())[0]  
cells_x_010=np.array(basin_lat_010)[:]
cells_y_010=np.array(basin_lon_010)[:]
mascara=np.array(basin.variables[var_type][:,:])
mask=mascara!=mascara[1,1]
mask=np.flip(mask*1,axis=0).astype('float64')
mask[mask==0]=np.nan
mask_st=mask.reshape((1, mask.shape[0],mask.shape[1]))
mask_basin_010=mask_st.reshape(mask_st.shape[1],mask_st.shape[2])
basin.close()
# Find coordinates where basin exists in mask
basin_coords_010=np.where(mask_basin_010==1)
# Find position of coordinates in datasets arrays
basin_true_longs_010=basin_lon_010.data[basin_coords_010[1]]+360
basin_true_lats_010=np.flip(basin_lat_010.data)[basin_coords_010[0]]

#### **ENSO**

In [ ]:
oni=pd.read_csv('Data/Magdalena/ENSO/ONI.csv',sep=';')
oni_nina=oni[oni['ONI']<=-0.5].reset_index() # Niña
oni_nino=oni[oni['ONI']>=0.5].reset_index() # Niño

#### **Elevation**

In [ ]:
nc_path="Data/Magdalena/Reanalysis/ERA5/Geopotential/geopotential.nc"
nc_climatic = nc.Dataset(nc_path)
geopot_lat=nc_climatic.variables['latitude'][:]
geopot_lon=nc_climatic.variables['longitude'][:]

## 2. Compute 3D Objects Properties

#### **Local Data NetCDF(IDW) - 0.10**

In [ ]:
# Filter to basin
# lat, long and basin
nc_lat=np.round(nc_lat_idw010.data,1)
nc_lon=np.round(nc_lon_idw010.data,1)
basin_lat=np.round(basin_lat_010.data,1).astype('float32')
basin_lon=np.round(basin_lon_010.data,1).astype('float32')+360
all_lat=np.intersect1d(basin_lat,nc_lat)
all_lon=np.intersect1d(basin_lon,nc_lon)
all_lat_mod=all_lat[::-1]
all_lon_mod=all_lon.copy()
dtset=idw10
mascara=mask_basin_010
name='IDW10'
if nc_lat[0]>nc_lat[1]:
    # Lat max
    lat_max_dataset=np.where((nc_lat==np.min(all_lat)))[0][0]
    # Lat min
    lat_min_dataset=np.where((nc_lat==np.max(all_lat)))[0][0]     
else:
    # Lat min
    lat_min_dataset=np.where((nc_lat==np.min(all_lat)))[0][0]
    # Lat max
    lat_max_dataset=np.where((nc_lat==np.max(all_lat)))[0][0]
lat_min_basin=np.where((np.flip(basin_lat)==np.max(all_lat)))[0][0]
lat_max_basin=np.where((np.flip(basin_lat)==np.min(all_lat)))[0][0]
# Lon min
lon_min_basin=np.where((basin_lon==np.min(all_lon)))[0][0]
lon_min_dataset=np.where((nc_lon==np.min(all_lon)))[0][0]
# Lon max
lon_max_basin=np.where((basin_lon==np.max(all_lon)))[0][0]
lon_max_dataset=np.where((nc_lon==np.max(all_lon)))[0][0]
# Filtered
mask_filt=mascara[lat_min_basin:lat_max_basin+1,lon_min_basin:lon_max_basin+1]
dtset_filt=dtset[:,lat_min_dataset:lat_max_dataset+1,lon_min_dataset:lon_max_dataset+1]
data_filter=dtset_filt*mask_filt

In [ ]:
plt.imshow(mask_filt)

In [ ]:
print(np.nanmax(data_filter))
print(np.nanmean(data_filter))
print(np.nanstd(data_filter))

In [ ]:
# Elevation
lat_args=np.sort(np.array([np.where(geopot_lat==i)[0][0] for i in basin_lat_010[lat_min_basin:lat_max_basin+1]]))
lon_args=np.sort(np.array([np.where(geopot_lon==i)[0][0] for i in basin_lon_010[lon_min_basin:lon_max_basin+1]+360]))
elevation=np.array(nc_climatic['z'][:,lat_args,lon_args]/9.80665)[0]

In [ ]:
# Object Threshold (what is classified as a precipitation)
threshold=1 
data_th=data_filter*(data_filter>=threshold)
# Connected component label (find different rainfall objects)
labels, num_labels = measure.label(data_th, connectivity=2, return_num=True,background=0) 
# Scikit-image (find object properties)
properties = ['label', 'area', 'centroid','bbox'] # ['area','extent','solidity','eccentricity','orientation','label','centroid'] 
props=pd.DataFrame(measure.regionprops_table(labels, properties=properties)) #
props['Total time']=props['bbox-3']-props['bbox-0']
props['start date']=[time_his[i] for i in props['bbox-0']]
props['end date']=[time_his[i-1] for i in props['bbox-3']]

In [ ]:
##### NIÑA EVENTS
print('Filter by Season')
# Filter by Total time
props_oni=np.nan
props_oni=props[props['Total time']>1].reset_index()
# Filter by Niña eventss
props_oni['ONI_nina']=np.nan
for k in range(len(props_oni)):
    if ((oni_nina['Year']==props_oni['start date'][k].year) & (oni_nina['Month']==props_oni['start date'][k].month)).any():
        props_oni['ONI_nina'][k]=1
    elif ((oni_nina['Year']==props_oni['end date'][k].year) & (oni_nina['Month']==props_oni['end date'][k].month)).any():
        props_oni['ONI_nina'][k]=1
    else:
        props_oni['ONI_nina'][k]=0
props_oni=props_oni[props_oni['ONI_nina']==1].reset_index()
# Pre-allocate other variables
props_oni=props_oni.drop(0)
props_oni=props_oni.sort_values("area",ascending=False,ignore_index=True)
# Number of events and distribution
print('Number of events: '+str(props_oni.shape[0]))
print(props_oni.area.quantile([0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1]))
props_oni=props_oni.loc[:1000] # Filter only 1000 biggest events
print('Compute parameters')
props_oni['Precipitation volume']=np.nan
props_oni['Precipitation max magnitude']=np.nan
props_oni['Precipitation max date']=np.nan
props_oni['Average Centroid lat']=np.nan
props_oni['Average Centroid long']=np.nan
props_oni['Start Centroid lat']=np.nan
props_oni['Start Centroid long']=np.nan
props_oni['Start Centroid elev']=np.nan
props_oni['Start average elev']=np.nan
props_oni['Finish Centroid lat']=np.nan
props_oni['Finish Centroid long']=np.nan
props_oni['Finish Centroid elev']=np.nan
props_oni['Finish average elev']=np.nan
conv=111*1000
area=(conv*0.10)**2
# Loop for nina events
for j in range(len(props_oni)): # len(props_oni)
    x,y,z=np.where(labels==props_oni['label'][j])
    precip_object=data_th[x,y,z]
    # Precipitation volume and max
    props_oni['Precipitation volume'].loc[j]=(area*np.sum(precip_object)/1000)/10**6
    props_oni['Precipitation max magnitude'].loc[j]=np.max(precip_object)
    props_oni['Precipitation max date'].loc[j]=time_his[x[np.argmax(precip_object)]]
    # Centroid Mean
    yc=props_oni['centroid-2'].loc[j] 
    xc=props_oni['centroid-1'].loc[j]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Average Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Average Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    # Centroid Start
    lab_cond=labels==props_oni['label'].loc[j] 
    lab_id=np.where(lab_cond)[0][0]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_start=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_start['centroid-1'].loc[0] 
    xc=props_start['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Start Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Start Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Start Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))] # Centroid elevation
    props_oni['Start average elev'][j]=elevation[lab_all.astype(bool)].mean()
    # Centroid Finish
    lab_cond=labels==props_oni['label'][j] 
    lab_id=np.where(lab_cond)[0][-1]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_finish=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_finish['centroid-1'].loc[0] 
    xc=props_finish['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Finish Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Finish Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Finish Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))]  # Centroid elevation
    props_oni['Finish average elev'][j]=elevation[lab_all.astype(bool)].mean()
props_oni=props_oni.drop(columns=["level_0","index"])
props_oni.to_csv('Results/Volume Aggregation/'+name+'_daily_nina2.csv')

In [ ]:
##### NIÑO EVENTS
print('Filter by Season')
# Filter by Total time
props_oni=np.nan
props_oni=props[props['Total time']>1].reset_index()
# Filter by Niña events
props_oni['ONI_nino']=np.nan
for k in range(len(props_oni)):
    if ((oni_nino['Year']==props_oni['start date'][k].year) & (oni_nino['Month']==props_oni['start date'][k].month)).any():
        props_oni['ONI_nino'][k]=1
    elif ((oni_nino['Year']==props_oni['end date'][k].year) & (oni_nino['Month']==props_oni['end date'][k].month)).any():
        props_oni['ONI_nino'][k]=1
    else:
        props_oni['ONI_nino'][k]=0
props_oni=props_oni[props_oni['ONI_nino']==1].reset_index()
# Pre-allocate other variables
props_oni=props_oni.drop(0)
props_oni=props_oni.sort_values("area",ascending=False,ignore_index=True)
# Number of events and distribution
print('Number of events: '+str(props_oni.shape[0]))
print(props_oni.area.quantile([0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1]))
props_oni=props_oni.loc[:1000] # Filter only 1000 biggest events
print('Compute parameters')
props_oni['Precipitation volume']=np.nan
props_oni['Precipitation max magnitude']=np.nan
props_oni['Precipitation max date']=np.nan
props_oni['Average Centroid lat']=np.nan
props_oni['Average Centroid long']=np.nan
props_oni['Start Centroid lat']=np.nan
props_oni['Start Centroid long']=np.nan
props_oni['Start Centroid elev']=np.nan
props_oni['Start average elev']=np.nan
props_oni['Finish Centroid lat']=np.nan
props_oni['Finish Centroid long']=np.nan
props_oni['Finish Centroid elev']=np.nan
props_oni['Finish average elev']=np.nan
conv=111*1000
area=(conv*0.10)**2
# Loop for nina events
for j in range(len(props_oni)): # len(props_oni)
    x,y,z=np.where(labels==props_oni['label'][j])
    precip_object=data_th[x,y,z]
    # Precipitation volume and max
    props_oni['Precipitation volume'].loc[j]=(area*np.sum(precip_object)/1000)/10**6
    props_oni['Precipitation max magnitude'].loc[j]=np.max(precip_object)
    props_oni['Precipitation max date'].loc[j]=time_his[x[np.argmax(precip_object)]]
    # Centroid Mean
    yc=props_oni['centroid-2'].loc[j] 
    xc=props_oni['centroid-1'].loc[j]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Average Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Average Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    # Centroid Start
    lab_cond=labels==props_oni['label'].loc[j] 
    lab_id=np.where(lab_cond)[0][0]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_start=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_start['centroid-1'].loc[0] 
    xc=props_start['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Start Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Start Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Start Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))] # Centroid elevation
    props_oni['Start average elev'][j]=elevation[lab_all.astype(bool)].mean()
    # Centroid Finish
    lab_cond=labels==props_oni['label'][j] 
    lab_id=np.where(lab_cond)[0][-1]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_finish=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_finish['centroid-1'].loc[0] 
    xc=props_finish['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Finish Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Finish Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Finish Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))]  # Centroid elevation
    props_oni['Finish average elev'][j]=elevation[lab_all.astype(bool)].mean()
props_oni=props_oni.drop(columns=["level_0","index"])
props_oni.to_csv('Results/Volume Aggregation/'+name+'_daily_nino2.csv')

#### **Eraland**

In [ ]:
# Filter to basin
# lat, long and basin
nc_lat=np.round(nc_lat_eraland.data,1)
nc_lon=np.round(nc_lon_eraland.data,1)
basin_lat=np.round(basin_lat_010.data,1).astype('float32')
basin_lon=np.round(basin_lon_010.data,1).astype('float32')+360
all_lat=np.intersect1d(basin_lat,nc_lat.data)
all_lon=np.intersect1d(basin_lon,nc_lon.data)
all_lat_mod=all_lat[::-1]
all_lon_mod=all_lon.copy()
dtset=eraland
mascara=mask_basin_010
name='eraland'
if nc_lat[0]>nc_lat[1]:
    # Lat max
    lat_max_dataset=np.where((nc_lat==np.min(all_lat)))[0][0]
    # Lat min
    lat_min_dataset=np.where((nc_lat==np.max(all_lat)))[0][0]     
else:
    # Lat min
    lat_min_dataset=np.where((nc_lat==np.min(all_lat)))[0][0]
    # Lat max
    lat_max_dataset=np.where((nc_lat==np.max(all_lat)))[0][0]
lat_min_basin=np.where((np.flip(basin_lat)==np.max(all_lat)))[0][0]
lat_max_basin=np.where((np.flip(basin_lat)==np.min(all_lat)))[0][0]
# Lon min
lon_min_basin=np.where((basin_lon==np.min(all_lon)))[0][0]
lon_min_dataset=np.where((nc_lon==np.min(all_lon)))[0][0]
# Lon max
lon_max_basin=np.where((basin_lon==np.max(all_lon)))[0][0]
lon_max_dataset=np.where((nc_lon==np.max(all_lon)))[0][0]
# Filtered
mask_filt=mascara[lat_min_basin:lat_max_basin+1,lon_min_basin:lon_max_basin+1]
dtset_filt=dtset[:,lat_min_dataset:lat_max_dataset+1,lon_min_dataset:lon_max_dataset+1]
data_filter=dtset_filt*mask_filt
# Replace no data with nan
data_filter[data_filter<0]=np.nan

In [ ]:
plt.imshow(mask_filt)

In [ ]:
print(np.nanmax(data_filter))
print(np.nanmean(data_filter))
print(np.nanstd(data_filter))

In [ ]:
# Elevation
lat_args=np.sort(np.array([np.where(geopot_lat==i)[0][0] for i in basin_lat_010[lat_min_basin:lat_max_basin+1]]))
lon_args=np.sort(np.array([np.where(geopot_lon==i)[0][0] for i in basin_lon_010[lon_min_basin:lon_max_basin+1]+360]))
elevation=np.array(nc_climatic['z'][:,lat_args,lon_args]/9.80665)[0]

In [ ]:
# Object Threshold (what is classified as a precipitation)
threshold=1
data_th=data_filter*(data_filter>=threshold)
# Connected component label (find different rainfall objects)
labels, num_labels = measure.label(data_th, connectivity=2, return_num=True,background=0) 
# Scikit-image (find object properties)
properties = ['label', 'area', 'centroid','bbox'] # ['area','extent','solidity','eccentricity','orientation','label','centroid'] 
props=pd.DataFrame(measure.regionprops_table(labels, properties=properties)) 
props['Total time']=props['bbox-3']-props['bbox-0']
props['start date']=[time_his[i] for i in props['bbox-0']]
props['end date']=[time_his[i-1] for i in props['bbox-3']]

In [ ]:
##### NIÑA EVENTS
print('Filter by Season')
# Filter by Total time
props_oni=np.nan
props_oni=props[props['Total time']>1].reset_index()
# Filter by Niña events
props_oni['ONI_nina']=np.nan
for k in range(len(props_oni)):
    if ((oni_nina['Year']==props_oni['start date'][k].year) & (oni_nina['Month']==props_oni['start date'][k].month)).any():
        props_oni['ONI_nina'][k]=1
    elif ((oni_nina['Year']==props_oni['end date'][k].year) & (oni_nina['Month']==props_oni['end date'][k].month)).any():
        props_oni['ONI_nina'][k]=1
    else:
        props_oni['ONI_nina'][k]=0
props_oni=props_oni[props_oni['ONI_nina']==1].reset_index()
# Pre-allocate other variables
props_oni=props_oni.drop(0)
props_oni=props_oni.sort_values("area",ascending=False,ignore_index=True)
# Number of events and distribution
print('Number of events: '+str(props_oni.shape[0]))
print(props_oni.area.quantile([0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1]))
props_oni=props_oni.loc[:1000] # Filter only 1000 biggest events
print('Compute parameters')
props_oni['Precipitation volume']=np.nan
props_oni['Precipitation max magnitude']=np.nan
props_oni['Precipitation max date']=np.nan
props_oni['Average Centroid lat']=np.nan
props_oni['Average Centroid long']=np.nan
props_oni['Start Centroid lat']=np.nan
props_oni['Start Centroid long']=np.nan
props_oni['Start Centroid elev']=np.nan
props_oni['Start average elev']=np.nan
props_oni['Finish Centroid lat']=np.nan
props_oni['Finish Centroid long']=np.nan
props_oni['Finish Centroid elev']=np.nan
props_oni['Finish average elev']=np.nan
conv=111*1000
area=(conv*0.10)**2
# Loop for nina events
for j in range(len(props_oni)): # len(props_oni)
    x,y,z=np.where(labels==props_oni['label'][j])
    precip_object=data_th[x,y,z]
    # Precipitation volume and max
    props_oni['Precipitation volume'].loc[j]=(area*np.sum(precip_object)/1000)/10**6
    props_oni['Precipitation max magnitude'].loc[j]=np.max(precip_object)
    props_oni['Precipitation max date'].loc[j]=time_his[x[np.argmax(precip_object)]]
    # Centroid Mean
    yc=props_oni['centroid-2'].loc[j] 
    xc=props_oni['centroid-1'].loc[j]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Average Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Average Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    # Centroid Start
    lab_cond=labels==props_oni['label'].loc[j] 
    lab_id=np.where(lab_cond)[0][0]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_start=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_start['centroid-1'].loc[0] 
    xc=props_start['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Start Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Start Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Start Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))] # Centroid elevation
    props_oni['Start average elev'][j]=elevation[lab_all.astype(bool)].mean()
    # Centroid Finish
    lab_cond=labels==props_oni['label'][j] 
    lab_id=np.where(lab_cond)[0][-1]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_finish=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_finish['centroid-1'].loc[0] 
    xc=props_finish['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Finish Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Finish Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Finish Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))]  # Centroid elevation
    props_oni['Finish average elev'][j]=elevation[lab_all.astype(bool)].mean()
props_oni=props_oni.drop(columns=["level_0","index"])
props_oni.to_csv('Results/Volume Aggregation/'+name+'_daily_nina2.csv')

In [ ]:
##### NIÑO EVENTS
print('Filter by Season')
# Filter by Total time
props_oni=np.nan
props_oni=props[props['Total time']>1].reset_index()
# Filter by Niña events
props_oni['ONI_nino']=np.nan
for k in range(len(props_oni)):
    if ((oni_nino['Year']==props_oni['start date'][k].year) & (oni_nino['Month']==props_oni['start date'][k].month)).any():
        props_oni['ONI_nino'][k]=1
    elif ((oni_nino['Year']==props_oni['end date'][k].year) & (oni_nino['Month']==props_oni['end date'][k].month)).any():
        props_oni['ONI_nino'][k]=1
    else:
        props_oni['ONI_nino'][k]=0
props_oni=props_oni[props_oni['ONI_nino']==1].reset_index()
# Pre-allocate other variables
props_oni=props_oni.drop(0)
props_oni=props_oni.sort_values("area",ascending=False,ignore_index=True)
# Number of events and distribution
print('Number of events: '+str(props_oni.shape[0]))
print(props_oni.area.quantile([0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1]))
props_oni=props_oni.loc[:1000] # Filter only 1000 biggest events
print('Compute parameters')
props_oni['Precipitation volume']=np.nan
props_oni['Precipitation max magnitude']=np.nan
props_oni['Precipitation max date']=np.nan
props_oni['Average Centroid lat']=np.nan
props_oni['Average Centroid long']=np.nan
props_oni['Start Centroid lat']=np.nan
props_oni['Start Centroid long']=np.nan
props_oni['Start Centroid elev']=np.nan
props_oni['Start average elev']=np.nan
props_oni['Finish Centroid lat']=np.nan
props_oni['Finish Centroid long']=np.nan
props_oni['Finish Centroid elev']=np.nan
props_oni['Finish average elev']=np.nan
conv=111*1000
area=(conv*0.10)**2
# Loop for nina events
for j in range(len(props_oni)): # len(props_oni)
    x,y,z=np.where(labels==props_oni['label'][j])
    precip_object=data_th[x,y,z]
    # Precipitation volume and max
    props_oni['Precipitation volume'].loc[j]=(area*np.sum(precip_object)/1000)/10**6
    props_oni['Precipitation max magnitude'].loc[j]=np.max(precip_object)
    props_oni['Precipitation max date'].loc[j]=time_his[x[np.argmax(precip_object)]]
    # Centroid Mean
    yc=props_oni['centroid-2'].loc[j] 
    xc=props_oni['centroid-1'].loc[j]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Average Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Average Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    # Centroid Start
    lab_cond=labels==props_oni['label'].loc[j] 
    lab_id=np.where(lab_cond)[0][0]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_start=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_start['centroid-1'].loc[0] 
    xc=props_start['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Start Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Start Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Start Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))] # Centroid elevation
    props_oni['Start average elev'][j]=elevation[lab_all.astype(bool)].mean()
    # Centroid Finish
    lab_cond=labels==props_oni['label'][j] 
    lab_id=np.where(lab_cond)[0][-1]
    lab_all=lab_cond[lab_id,:,:].astype(np.uint8)
    props_finish=pd.DataFrame(measure.regionprops_table(lab_all, properties=['label','centroid']))
    yc=props_finish['centroid-1'].loc[0] 
    xc=props_finish['centroid-0'].loc[0]
    xp1=[np.floor(xc),np.ceil(xc)]
    yp1=[all_lat_mod[np.floor(xc).astype(int)],all_lat_mod[np.ceil(xc).astype(int)]]
    xp2=[np.floor(yc),np.ceil(yc)]
    yp2=[all_lon_mod[np.floor(yc).astype(int)],all_lon_mod[np.ceil(yc).astype(int)]]
    props_oni['Finish Centroid lat'][j]=np.interp(xc, xp1, yp1)
    props_oni['Finish Centroid long'][j]=np.interp(yc, xp2, yp2)-360
    props_oni['Finish Centroid elev'][j]=elevation[int(np.round(xc)),int(np.round(yc))]  # Centroid elevation
    props_oni['Finish average elev'][j]=elevation[lab_all.astype(bool)].mean()
props_oni=props_oni.drop(columns=["level_0","index"])
props_oni.to_csv('Results/Volume Aggregation/'+name+'_daily_nino2.csv')